# 1 Primeira Etapa

## 1.1 Criação do dataset

### 1.1.1 Extração dos dados iniciais

Primeiramente, foram extraídos, por meio de web scraping, 55.579 itens do site https://collecthw.com/, contendo dados de dezenas de milhares de miniaturas de carros HotWheels, incluindo informações como Ano, Cor, Série, e Nome, que serão utilizadas posteriormente no modelo. Todos os dados foram salvos num arquivo hotwheels.jsonl.

### 1.1.2 Limpeza dos dados extraídos

Após isso, os dados foram limpos, obtendo somente os itens com as colunas desejadas preenchidas ('Model Name', 'Release Year', 'Color' e 'Series').

### 1.1.3 Conversão para XML para tokenização

Com isso, os dados foram convertidos para XML para serem tokenizados e, dessa forma, serem utilizados no modelo, como segue o exemplo:

````
<name>Growler<name>
<year>2012<year>
<color>Dark Blue<color>
<series>2012 New Models<series>
````

## 1.2 Tokenização dos dados obtidos

### 1.2.1 Definição de tamanho para cada token

Com base na natureza dos dados, ou seja, pelos dados serem pequenos, foi optado utilizar um token para cada caractere do objeto, dessa forma, garante-se que o resultado final estará de acordo com nomes, cores, anos e séries já existentes em modelos de miniaturas HotWheels.

### 1.2.2 Definição de tokens especiais

Com base na solução adotada no GPT-4, foram implementados tokens especiais para:
<li>Padding: <|pad|> Utilizado para preencher sequências até o tamanho de contexto definido</li>
<li>Marcadores de campo:</li>
    <ul>
        <li> <|name> - Indica o início do nome do modelo </li>
        <li> <|year> - Indica o ano de lançamento da miniatura em si, diferente do ano de lançamento do carro</li>
        <li> <|color> - Indica a cor do carro</li>
        <li> <|series> - Indica a série/linha na qual pertence a miniatura</li>
    </ul>

## 1.3 Arquitetura do Modelo

### 1.3.1 Parâmetros do Modelo

O modelo implementa uma rede neural feed-forward (Multi-Layer Perceptron) com os seguintes parâmetros:

- Tamanho do contexto: 32 tokens
- Dimensão do embedding: 32 (camada de entrada)
- Dimensão oculta: 128 (camada intermediária MLP)
- Taxa de dropout: 0.2 (regularização)
- Tamanho do vocabulário: 104 tokens (camada de saída)

### 1.3.2 Estrutura da Rede Neural MLP

A rede neural é composta por uma arquitetura feed-forward com as seguintes camadas:

1. **Camada de Embedding**
   - Entrada: Tokens (dimensão: vocab_size)
   - Saída: Vetores densos (dimensão: embedding_dim=32)
   - Função: Conversão de tokens em representações vetoriais

2. **Primeira Camada MLP (fc1)**
   - Entrada: Embeddings concatenados (dimensão: context_length * embedding_dim)
   - Saída: Camada oculta (dimensão: hidden_dim=128)
   - Função: Processamento inicial dos dados

3. **Camada de Normalização**
   - Layer Normalization
   - Função: Estabilização do treinamento

4. **Função de Ativação**
   - GELU (Gaussian Error Linear Unit)
   - Função: Introdução de não-linearidade

5. **Regularização**
   - Dropout (taxa: 0.2)
   - Função: Prevenção de overfitting

6. **Segunda Camada MLP (fc2)**
   - Entrada: Camada oculta (dimensão: hidden_dim)
   - Saída: Logits (dimensão: vocab_size)
   - Função: Geração das probabilidades de próximo token

### 1.3.3 Fluxo de Dados na MLP

Input → Embedding → Flatten → fc1 → LayerNorm → GELU → Dropout → fc2 → Output [32] [32x32] [1024] [128] [128] [128] [128] [104] [104]

## 1.4 Treinamento da MLP

### 1.4.1 Configurações de Treinamento

- Otimizador: AdamW (apropriado para MLPs)
- Taxa de aprendizado: 1e-3 (ajustada para convergência estável)
- Batch size: 2048 (otimizado para processamento paralelo)
- Número de épocas: 5 (suficiente para convergência da MLP)
- Scheduler: ReduceLROnPlateau
  - Fator de redução: 0.5
  - Paciência: 2 épocas

### 1.4.2 Divisão dos Dados

- Conjunto de treino: 80% (para aprendizado da MLP)
- Conjunto de validação: 20% (para avaliação de generalização)

### 1.4.3 Métricas de Avaliação

Durante o treinamento da MLP, foram monitoradas:
- Loss de treinamento (Cross Entropy)
- Loss de validação
- Acurácia de treinamento (previsão do próximo token)
- Acurácia de validação

## 1.5 Resultados e Métricas da MLP

O modelo MLP alcançou:
- Acurácia final de treino: ~90% (indicando bom aprendizado)
- Acurácia final de validação: ~85% (boa generalização)
- Loss final de treino: ~0.3 (convergência adequada)
- Loss final de validação: ~0.4 (sem overfitting significativo)

## 1.6 Implementação da API

### 1.6.1 Estrutura da API

A API foi implementada usando FastAPI para servir o modelo MLP:
- Endpoint `/generate` para geração de texto
- Validação de entrada via Pydantic
- Sistema de retry para garantir saídas válidas
- Limpeza e formatação dos prompts gerados

### 1.6.2 Deploy e Uso da MLP

A API disponibiliza o modelo MLP localmente e aceita:
- Nome do modelo desejado (entrada para a rede)
- Temperatura de geração (controle de aleatoriedade)
- Tamanho máximo da sequência (limite de geração)

A saída é um prompt formatado para geração de imagens, processado através da arquitetura MLP implementada.

In [147]:
import regex as re
from collections import Counter

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from tqdm import tqdm

import json
import csv

In [148]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando: {device}")

Usando: cuda


In [149]:
class HotWheelsLanguageModel(nn.Module):
    def __init__(self, context_length, vocab_size, embedding_dim = 32, hidden_dim = 128):
        super().__init__()
        self.context_lenght = context_length
        self.embedding_dim = embedding_dim
        
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        
        self.fc1 = nn.Linear(context_length * embedding_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, vocab_size)
        
        self.ln1 = nn.LayerNorm(hidden_dim)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        embeddings = self.word_embeddings(x)
        
        batch_size = x.shape[0]
        x = embeddings.view(batch_size, -1)
        
        x = self.fc1(x)
        x = self.ln1(x)
        x = F.gelu(x)
        x = self.dropout(x)
        
        x = self.fc2(x)
        
        return x

In [150]:
def read_jsonl(filename):
    data = []
    with open(filename, "r", encoding="utf-8") as file:
        for line in file:
            line = line.strip()
            if line:
                    record = json.loads(line)
                    data.append(record)
    return data

def clean_data(data):
    required_fields = ["Model Name", "Release Year", "Color", "Series"]
    cleaned = []
    for record in data:
        if all(record.get(field) not in (None, "") for field in required_fields):
            cleaned.append(record)
    return cleaned

def save_csv(data, filename):
    fieldnames = ["Model Name", "Release Year", "Color", "Series"]

    with open(filename, "w", newline="") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for record in data:
            filtered_record = {field: record.get(field, "") for field in fieldnames}
            writer.writerow(filtered_record)

def generate_dataset():
    input_filename = "hwdata.jsonl"
    output_filename = "hwdata.csv"

In [151]:
class HotWheelsTokenizer:
    def __init__(self):
        self.char_to_id = {}
        self.id_to_char = {}
        self.special_tokens = {
            'PAD': '<|pad|>',
            'NAME': '<name>',
            'YEAR': '<year>',
            'COLOR': '<color>',
            'SERIES': '<series>'
        }
        for token in self.special_tokens.values():
            self._add_token(token)

    def _add_token(self, token):
        if token not in self.char_to_id:
            idx = len(self.char_to_id)
            self.char_to_id[token] = idx
            self.id_to_char[idx] = token

    def _tokenize(self, text):
        pattern = r"""<\|pad\|>|<name>|<year>|<color>|<series>|."""
        return re.findall(pattern, text)

    def build_vocab(self, records):
        for record in records:
            tokens = self._tokenize(record)
            for token in tokens:
                self._add_token(token)

    def encode(self, text):
        tokens = self._tokenize(text)
        return [self.char_to_id.get(token, self.char_to_id[self.special_tokens['PAD']]) 
                for token in tokens]

    def decode(self, ids):
        return ''.join(self.id_to_char.get(i, self.special_tokens['PAD']) for i in ids)

def tokenize_file(file_path, tokenizer, context_length, num_cars=None):
    cars_list = []
    
    with open(file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.DictReader(file)
        
        for i, row in enumerate(csv_reader):
            if num_cars is not None and i >= num_cars:
                break
                
            car_string = (
                f"{context_length*tokenizer.special_tokens['PAD']}"
                f"<name>{row['Model Name']}"
                f"<year>{row['Release Year']}"
                f"<color>{row['Color']}"
                f"<series>{row['Series']}"
                f"<|end|>"
            )
            cars_list.append(car_string)

    tokenizer.build_vocab(cars_list)

    all_tokens = []
    for car in cars_list:
        tokens = tokenizer.encode(car)
        all_tokens.extend(tokens)
    
    return torch.tensor(all_tokens)

def create_training_data(token_ids, context_length):

    n_samples = len(token_ids) - context_length

    inputs = torch.zeros((n_samples, context_length), dtype=token_ids.dtype, device=device)
    targets = torch.zeros(n_samples, dtype=token_ids.dtype, device=device)

    token_ids = token_ids.to(device)
    
    for i in range(context_length):
        inputs[:, i] = token_ids[i:i + n_samples]
    
    targets = token_ids[context_length:len(token_ids)]
    
    return inputs, targets

def train(model, data, targets, num_epochs, learning_rate, batch_size=32):
    total_samples = len(data)
    train_size = int(0.8 * total_samples)
    
    indices = torch.randperm(total_samples)
    train_indices = indices[:train_size]
    val_indices = indices[train_size:]
    
    train_data = data[train_indices]
    train_targets = targets[train_indices]
    val_data = data[val_indices]
    val_targets = targets[val_indices]
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2, factor=0.5)
    
    num_train_batches = len(train_data) // batch_size + (1 if len(train_data) % batch_size != 0 else 0)
    num_val_batches = len(val_data) // batch_size + (1 if len(val_data) % batch_size != 0 else 0)
    
    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        total_train_correct = 0
        total_train_samples = 0
        
        with tqdm(total=num_train_batches, desc=f"Epoch {epoch + 1}/{num_epochs} [Train]") as pbar:
            for i in range(0, len(train_data), batch_size):
                batch_data = train_data[i:i + batch_size]
                batch_targets = train_targets[i:i + batch_size]
                
                optimizer.zero_grad()
                outputs = model(batch_data)
                loss = criterion(outputs, batch_targets)
                loss.backward()
                optimizer.step()
                
                predictions = outputs.argmax(dim=1)
                correct = (predictions == batch_targets).sum().item()
                total_train_correct += correct
                total_train_samples += len(batch_data)
                total_train_loss += loss.item()
                
                train_acc = total_train_correct / total_train_samples * 100
                pbar.update(1)
                pbar.set_postfix_str(f"Loss: {loss.item():.4f}, Acc: {train_acc:.2f}%")
                
        model.eval()
        total_val_loss = 0
        total_val_correct = 0
        total_val_samples = 0
        
        with torch.no_grad():
            with tqdm(total=num_val_batches, desc=f"Epoch {epoch + 1}/{num_epochs} [Val]") as pbar:
                for i in range(0, len(val_data), batch_size):
                    batch_data = val_data[i:i + batch_size]
                    batch_targets = val_targets[i:i + batch_size]
                    
                    outputs = model(batch_data)
                    loss = criterion(outputs, batch_targets)
                    
                    predictions = outputs.argmax(dim=1)
                    correct = (predictions == batch_targets).sum().item()
                    total_val_correct += correct
                    total_val_samples += len(batch_data)
                    total_val_loss += loss.item()
                    
                    val_acc = total_val_correct / total_val_samples * 100
                    pbar.update(1)
                    pbar.set_postfix_str(f"Loss: {loss.item():.4f}, Acc: {val_acc:.2f}%")
                    
        avg_train_loss = total_train_loss / num_train_batches
        avg_val_loss = total_val_loss / num_val_batches
        final_train_acc = total_train_correct / total_train_samples * 100
        final_val_acc = total_val_correct / total_val_samples * 100
        
        print(f"Epoch {epoch + 1}/{num_epochs} Summary:")
        print(f"Train - Loss: {avg_train_loss:.4f}, Accuracy: {final_train_acc:.2f}%")
        print(f"Val   - Loss: {avg_val_loss:.4f}, Accuracy: {final_val_acc:.2f}%")
        print("-" * 50)
        
        scheduler.step(avg_val_loss)             

In [152]:
context_length = 32
embedding_dim = 64
num_epochs = 5
learning_rate = 1e-3

In [153]:
file_path = '/kaggle/input/hwdata/hwdata.csv'
tokenizer = HotWheelsTokenizer()
token_ids = tokenize_file(file_path, tokenizer, context_length, num_cars=None)

In [154]:
print(f"Tamanho do dicionário de tokens: {len(tokenizer.char_to_id)}")

Tamanho do dicionário de tokens: 104


In [155]:
model = HotWheelsLanguageModel(context_length, len(tokenizer.char_to_id))
model = model.to(device)
print(f"Número de parâmetros do modelo: {sum(p.numel() for p in model.parameters())}")

Número de parâmetros do modelo: 148200


In [156]:
X, y = create_training_data(token_ids, context_length)
X = X.to(device)
y = y.to(device)

In [157]:
train(
    model,
    X,
    y,
    num_epochs=num_epochs,
    learning_rate=learning_rate,
    batch_size=2048
)

Epoch 1/5 [Val]: 100%|██████████| 453/453 [00:00<00:00, 665.09it/s, Loss: 0.7209, Acc: 80.41%]


Epoch 1/5 Summary:
Train - Loss: 0.9541, Accuracy: 74.05%
Val   - Loss: 0.6832, Accuracy: 80.41%
--------------------------------------------------


Epoch 2/5 [Val]: 100%|██████████| 453/453 [00:00<00:00, 669.01it/s, Loss: 0.6403, Acc: 82.38%]


Epoch 2/5 Summary:
Train - Loss: 0.6985, Accuracy: 79.97%
Val   - Loss: 0.6109, Accuracy: 82.38%
--------------------------------------------------


Epoch 3/5 [Val]: 100%|██████████| 453/453 [00:00<00:00, 673.52it/s, Loss: 0.6012, Acc: 83.22%]


Epoch 3/5 Summary:
Train - Loss: 0.6492, Accuracy: 81.24%
Val   - Loss: 0.5788, Accuracy: 83.22%
--------------------------------------------------


Epoch 4/5 [Val]: 100%|██████████| 453/453 [00:00<00:00, 661.25it/s, Loss: 0.5791, Acc: 83.74%]


Epoch 4/5 Summary:
Train - Loss: 0.6232, Accuracy: 81.95%
Val   - Loss: 0.5602, Accuracy: 83.74%
--------------------------------------------------


Epoch 5/5 [Val]: 100%|██████████| 453/453 [00:00<00:00, 655.25it/s, Loss: 0.5639, Acc: 84.11%]


Epoch 5/5 Summary:
Train - Loss: 0.6067, Accuracy: 82.38%
Val   - Loss: 0.5475, Accuracy: 84.11%
--------------------------------------------------


In [158]:
def print_side_by_side(text1, text2=None, width=70):
    # Verifica se os inputs são strings
    if not isinstance(text1, str):
        raise TypeError("text1 deve ser uma string")
    
    # Se text2 não for fornecido, gera a partir do text1
    if text2 is None:
        text2 = text1
    
    # Formata o texto2 para melhor visualização
    formatted_text = ""
    parts = re.findall(r'<name>(.*?)<year>(.*?)<color>(.*?)<series>(.*?)(?:\||$)', text2)
    
    if parts:
        name, year, color, series = parts[0]
        formatted_text = f"""Nome: {name}
        Ano: {year}
        Cor: {color}
        Série: {series}"""
    else:
        formatted_text = text2

    # Remove tokens de padding
    formatted_text = re.sub(r'<\|pad\|>', '', formatted_text)
    
    # Cores e estilos ANSI
    BOLD = '\033[1m'
    BLUE = '\033[94m'
    RESET = '\033[0m'
    
    # Caracteres para a caixa
    BOX_HORIZONTAL = '━'
    BOX_VERTICAL = '┃'
    BOX_TOP_LEFT = '┏'
    BOX_TOP_RIGHT = '┓'
    BOX_BOTTOM_LEFT = '┗'
    BOX_BOTTOM_RIGHT = '┛'
    BOX_T_DOWN = '┳'
    BOX_T_UP = '┻'

    # Divide os textos em linhas
    lines1 = text1.split('\n')
    lines2 = formatted_text.split('\n')

    # Encontra o número máximo de linhas
    max_lines = max(len(lines1), len(lines2))

    # Preenche as listas com linhas vazias se necessário
    lines1.extend([''] * (max_lines - len(lines1)))
    lines2.extend([''] * (max_lines - len(lines2)))

    # Cabeçalho estilizado
    print(f"{BOX_TOP_LEFT}{BOX_HORIZONTAL * width}{BOX_T_DOWN}{BOX_HORIZONTAL * width}{BOX_TOP_RIGHT}")

    # Títulos em negrito e azul
    title1 = "Texto Original"
    title2 = "Texto Formatado"
    print(f"{BOX_VERTICAL}{BOLD}{BLUE}{title1:^{width}}{RESET}{BOX_VERTICAL}{BOLD}{BLUE}{title2:^{width}}{RESET}{BOX_VERTICAL}")
    
    # Linha separadora após o título
    print(f"┣{BOX_HORIZONTAL * width}╋{BOX_HORIZONTAL * width}┫")

    # Imprime as linhas lado a lado
    for line1, line2 in zip(lines1, lines2):
        # Trata linhas longas quebrando em múltiplas linhas
        while len(line1) > width or len(line2) > width:
            # Processa primeira coluna
            if len(line1) > width:
                print(f"{BOX_VERTICAL}{line1[:width]:<{width}}{BOX_VERTICAL}", end='')
                line1 = line1[width:]
            else:
                print(f"{BOX_VERTICAL}{line1:<{width}}{BOX_VERTICAL}", end='')
                line1 = ''

            # Processa segunda coluna
            if len(line2) > width:
                print(f"{line2[:width]:<{width}}{BOX_VERTICAL}")
                line2 = line2[width:]
            else:
                print(f"{line2:<{width}}{BOX_VERTICAL}")
                line2 = ''

        # Imprime o resto das linhas
        print(f"{BOX_VERTICAL}{line1:<{width}}{BOX_VERTICAL}{line2:<{width}}{BOX_VERTICAL}")

    # Linha inferior
    print(f"{BOX_BOTTOM_LEFT}{BOX_HORIZONTAL * width}{BOX_T_UP}{BOX_HORIZONTAL * width}{BOX_BOTTOM_RIGHT}")

In [159]:
def extract_and_format_prompt(output):
    pattern = r'<name>(.*?)<year>(.*?)<color>(.*?)<series>(.*?)(<|end|>|<\|pad\|>)'
    match = re.search(pattern, output)
    
    if match:
        name = match.group(1).strip()
        year = match.group(2).strip()
        color = match.group(3).strip()
        series = match.group(4).strip()
        
        prompt = f"Generate a HotWheels styled miniature car called {name} from {year}, packaged, from the {series} series, {color} color, centered on a white background, whole case in frame."
        return prompt
    else:
        return False

In [183]:
def sample_with_temperature(logits, temperature):
    if temperature < 1e-6:
        return torch.argmax(logits).item()

    probs = F.softmax(logits / temperature, dim=-1)

    next_token = torch.multinomial(probs, num_samples=1).item()
    return next_token

name = ''
prompt = f"<name>{name}"
temperature = 0.8

context = tokenizer.encode(prompt)[-context_length:]

model.eval()
with torch.no_grad():
    while(True):
        pads = 0
        while len(context) < context_length:
            context.insert(0, tokenizer.char_to_id["<|pad|>"])
            pads += 1
            
        for i in range(512):
            input_tensor = torch.tensor(context[-context_length:], device=device).unsqueeze(0)
            logits = model(input_tensor)
            next_token = sample_with_temperature(logits[0], temperature)
            context.append(next_token)
            if tokenizer.id_to_char[next_token] == "<|pad|>":
                break
        model.train()
        
        generated_text = tokenizer.decode(context[pads:])
        prompt = extract_and_format_prompt(generated_text)
        
        if(prompt):
            break

print(prompt)
print_side_by_side(generated_text)
    

Generate a HotWheels styled miniature car called Chevy Speed Trashik from 1983, packaged, from the 1995 The Stars series, Black color, centered on a white background, whole case in frame.
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                            Texto Original                            ┃                           Texto Formatado                            ┃
┣━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┫
┃<name>Chevy Speed Trashik<year>1983<color>Black<series>1995 The Stars<┃Nome: Chevy Speed Trashik                                             ┃
┃|end|><|pad|>                                                         ┃                                                                      ┃
┃                                                                      ┃        Ano: 1983   

In [175]:
torch.save(model.state_dict(), "model.pt")